Basic demonstration of the quantum chemistry code I have written. It seems to be fairly quick, but uses quite a bit of ram. One trick which I certainly did not exploit is that some parts of the environments are simply the harmitian conjugate of other parts, and the environments can therefore be compressed further (and should be twice as cheap to calculate).

In [1]:
using Revise,LinearAlgebra, MPSKit, TensorKit, MPSKitExperimental

LinearAlgebra.BLAS.set_num_threads(1);
#TensorKit.TO.disable_cache();
TensorKit.Strided.disable_threads();


FCIDUMP is a well known (though badly documented) format to store the electronic integrals. I have written an experimental parser to load in such files:

In [3]:
(ERI,K,E0,NORB,NELEC,MS2) = parse_fcidump("N2.CCPVDZ.FCIDUMP")

([1.1486574428316065 + 0.0im 0.09862429942213358 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; 0.09862429942213358 + 0.0im 0.01772802215378584 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; … ; 0.0 + 0.0im 0.0 + 0.0im … 0.009036810489967113 + 0.0im 0.003653054088686124 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im … 0.003653054088686124 + 0.0im 0.0017533811663050644 + 0.0im;;; 0.09862429942213358 + 0.0im 0.413306086967678 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; 0.01772802215378584 + 0.0im -0.002611637416218654 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; … ; 0.0 + 0.0im 0.0 + 0.0im … -0.00533684879644213 + 0.0im 0.002227779206976413 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im … -0.001273700295269154 + 0.0im -0.0005596210416448279 + 0.0im;;; 0.03622757258934596 + 0.0im -0.0470908045818145 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; 0.0013749347300627281 + 0.0im 0.004411244674669814 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; … ; 0.0 + 0.0im 0.0 + 0.0im … -2.8000517192292658e-5 + 0.0im -0.006213347982843338 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im … -0.000871798277289064 + 

ERI is the Electronic Repulsion Integral, K is the two-body term, E0 is an energy shift, NORB is the number of orbitals (mps length), NELEC is the number of electrons and MS2 is the total spin.

fcidump files can contain orbital symmetries, but I ignore these.

In [4]:
(ham,_) = fused_quantum_chemistry_hamiltonian(E0,K,ERI,Float64);

The second return value contains some metadata which can be ignored. Instantiate the state:

In [5]:
psp = Vect[(Irrep[U₁]⊠Irrep[SU₂] ⊠ FermionParity)]((0,0,0)=>1, (1,1//2,1)=>1, (2,0,0)=>1);
left = Vect[(Irrep[U₁]⊠Irrep[SU₂] ⊠ FermionParity)]((-NELEC,MS2//2,mod(-NELEC,2))=>1);
right = oneunit(left);

virtual = Vect[(Irrep[U₁]⊠Irrep[SU₂] ⊠ FermionParity)]((i,s,b)=>1 for i in -NELEC:0, s in 0:1//2:(MS2//2+1), b in (0,1));

st = FiniteMPS(rand,Float64,NORB,psp,virtual;left,right);

We also need to instantiate the environments, because MPSKit will otherwise call the default environment constructor which keeps everything in ram (I think I even disabled that constructor). To create an environment that lives on disk, we can use:

In [6]:
envs = disk_environments(st,ham);

Let's call DMRG!

In [7]:
(st,envs) = find_groundstate(st,ham, DMRG2(trscheme=truncdim(50), verbosity = 2, maxiter=3),envs);

┌ Warning: DMRG2 cancel 3:	obj = -1.091292970502e+02	err = 2.1819949862e-03	time = 1.31 min
└ @ MPSKit /home/maarten/.julia/packages/MPSKit/nqYoZ/src/algorithms/groundstate/dmrg.jl:145
